[Large Movie Review Dataset](https://homl.info/imdb), which contains 50,000 movies reviews from the [Internet Movie Database](https://imdb.com/). The data is organized in two directories, `train` and `test`, each containing a `pos` subdirectory with 12,500 positive reviews and a `neg` subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders (including preprocessed bag-of-words), but we will ignore them in this exercise._

# Download Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import os

In [2]:
from pathlib import Path

DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = 'aclImdb_v1.tar.gz'
filepath  = tf.keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent / 'aclImdb'
path

84131840/84125825 [==============================] - 6s 0us/step


PosixPath('/root/.keras/datasets/aclImdb')

In [3]:
for name, subdirs, files in os.walk(path):
    indent = len(Path(name).parts) - len(path.parts)
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

aclImdb/
    README
    imdb.vocab
    imdbEr.txt
    train/
        labeledBow.feat
        unsupBow.feat
        urls_neg.txt
        ...
        unsup/
            0_0.txt
            10000_0.txt
            10001_0.txt
            ...
        neg/
            0_3.txt
            10000_4.txt
            10001_4.txt
            ...
        pos/
            0_9.txt
            10000_8.txt
            10001_10.txt
            ...
    test/
        labeledBow.feat
        urls_neg.txt
        urls_pos.txt
        neg/
            0_2.txt
            10000_4.txt
            10001_1.txt
            ...
        pos/
            0_10.txt
            10000_7.txt
            10001_9.txt
            ...


In [4]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_valid_pos), len(test_valid_neg)

(12500, 12500, 12500, 12500)

# Split the test set into validation set and test set

In [5]:
np.random.shuffle(test_valid_pos)

test_pos = test_valid_pos[:5000]
test_neg = test_valid_neg[:5000]
valid_pos = test_valid_pos[5000:]
valid_neg = test_valid_neg[5000:]

In [6]:
def imdb_dataset(filepaths_positive, filepaths_negative):
    reviews = []
    labels = []
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            with open(filepath) as review_file:
                reviews.append(review_file.read())
            labels.append(label)
    return tf.data.Dataset.from_tensor_slices(
        (tf.constant(reviews), tf.constant(labels)))

In [7]:
for X, y in imdb_dataset(train_pos, train_neg).take(3):
    print(X)
    print(y)
    print()

tf.Tensor(b'This is easily the most disappointing, least gratifying movie of the entire so-called blacksploitation genre, which, by the way, are films we generally enjoy a great deal in our home. Rather than being "exploitation" or demeaning, these films actually provide a priceless insight into an era. Well, not Bucktown.<br /><br />In this story, Duke returns to Bucktown to operate the night club left to him by his recently deceased brother. He quickly learns that the city is entirely controlled by a corrupt police force, bleeding protection money out of all the local businesses. Duke resists, and determines that he will rescue the city from the corrupt police. Unfortunately, he does so by calling in a posse of his friends (these people are vaguely explained as some former black-militants who have worked with Duke on "jobs" in the past) and they simply murder the entire police department in cold blood. And literally in the presence of hundreds of witnesses who do nothing to stop it. 

In [8]:
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).repeat(10): pass

1 loop, best of 1: 21 s per loop


It takes about 22 seconds to load dataset and go trough it 10 times

But let's pretend the dataset does not fit in memory, just to make things more interesting. Luckkily each review fits on just one line (they use `<br />` to indicate line breaks), so we can read the reviews using a `TextLineDataset`. If they didnt we would have to preprocess the input files. For very large datasets it would make sense to use a tool like Apache Beam for that

In [9]:
def imdb_dataset(filepaths_positive, filepaths_negative, n_read_threads = 5):
  dataset_neg = tf.data.TextLineDataset(filepaths_negative,
                                        num_parallel_reads = n_read_threads)
  dataset_neg = dataset_neg.map(lambda review: (review, 0))
  dataset_pos = tf.data.TextLineDataset(filepaths_positive,
                                        num_parallel_reads = n_read_threads)
  dataset_pos = dataset_pos.map(lambda review: (review, 1))

  return tf.data.Dataset.concatenate(dataset_pos, dataset_neg)

In [10]:
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).repeat(10): pass

1 loop, best of 1: 58.1 s per loop


Now it takes about 33 seconds to go through the dataset 10 times. That's much slower essentially because the dataset is not cached in RAM, so it must be reloaded at each epoch. If you add .cache() just before .repeat(10), you see that it will be as fast as previous one

In [11]:
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).cache().repeat(10): pass

1 loop, best of 1: 23.3 s per loop


In [12]:
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000).batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).shuffle(25000).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).shuffle(25000).batch(batch_size).prefetch(1)

Binary Classification model with `TextVectorization` layer to preprocess each review.

First preprocess the reviews cropping them to 300 characters, converting them to lower case and replacing <br/> and non letter characters to spaces

In [44]:
def preprocess(X_batch, n_words = 50):
  shape = tf.shape(X_batch) * tf.constant([1, 0 ]) + tf.constant([0, n_words])
  Z = tf.strings.substr(X_batch, 0, 300)
  Z = tf.strings.lower(Z)
  Z = tf.strings.regex_replace(Z, b"<br\\s*/?>", b" ")
  Z = tf.strings.regex_replace(Z, b"[^a-z]", b" ")
  Z = tf.strings.split(Z)
  return Z.to_tensor(shape = shape, default_value = b"<pad>")

X_example = tf.constant(["It's a great, great movie! I loved it.", "It was terrible, run away!!!"])
preprocess(X_example)

<tf.Tensor: shape=(2, 50), dtype=string, numpy=
array([[b'it', b's', b'a', b'great', b'great', b'movie', b'i', b'loved',
        b'it', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>'],
       [b'it', b'was', b'terrible', b'run', b'away', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'

Second utility function that will make a data sample with the same format as the output of the preprocess() function, and will output the list of the top max_size most frequent words, ensuring that the padding token is first

In [45]:
from collections import Counter

def get_vocabulary(data_sample, max_size = 1000):
  preprocessed_reviews = preprocess(data_sample).numpy()
  counter = Counter()
  for words in preprocessed_reviews:
    for word in words:
      if word != b'<pad>':
        counter[word] += 1
  return [b'<pad>'] + [word for word, count in counter.most_common(max_size)]

get_vocabulary(X_example)

[b'<pad>',
 b'it',
 b'great',
 b's',
 b'a',
 b'movie',
 b'i',
 b'loved',
 b'was',
 b'terrible',
 b'run',
 b'away']

Now we are ready to create a `TextVecorization` layer/ Its constructor just saves the hyperparameters (`max_vocabulary_size` and `n_oov_buckets`). The `adapt()` method computes the vocabulary using `get_vocabulary` fuction then it builds `StaticVocabularyTable`. The `call()` method preprocess the reviews to get a padded list of words for each review, then it uses the `StaticsVocabularyTable` to lookup the index of each word in vocabulary.

In [46]:
class TextVectorization(keras.layers.Layer):
    def __init__(self, max_vocabulary_size=1000, n_oov_buckets=100, dtype=tf.string, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.max_vocabulary_size = max_vocabulary_size
        self.n_oov_buckets = n_oov_buckets

    def adapt(self, data_sample):
        self.vocab = get_vocabulary(data_sample, self.max_vocabulary_size)
        words = tf.constant(self.vocab)
        word_ids = tf.range(len(self.vocab), dtype=tf.int64)
        vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
        self.table = tf.lookup.StaticVocabularyTable(vocab_init, self.n_oov_buckets)
        
    def call(self, inputs):
        preprocessed_inputs = preprocess(inputs)
        return self.table.lookup(preprocessed_inputs)

Lets try on our small `X_example` we defined earlier

In [47]:
text_vectorization = TextVecorization()

text_vectorization.adapt(X_example)
text_vectorization(X_example)

<tf.Tensor: shape=(2, 50), dtype=int64, numpy=
array([[ 1,  3,  4,  2,  2,  5,  6,  7,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 1,  8,  9, 10, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]])>

Looks good! As you can see, each review was cleaned up and tokenized, then each word was encoded as its index in the vocabulary (all the 0s correspond to the `<pad>` tokens).

Now let's create another `TextVectorization` layer and let's adapt it to the full IMDB training set (if the training set did not fit in RAM, we could just use a smaller sample of the training set by calling `train_set.take(500)`):

In [48]:
max_vocabulary_size = 1000
n_oov_buckets = 100

sample_review_batches = train_set.map(lambda review, label: review)
sample_reviews = np.concatenate(list(sample_review_batches.as_numpy_iterator()),
                                axis=0)

text_vectorization = TextVectorization(max_vocabulary_size, n_oov_buckets,
                                       input_shape=[])
text_vectorization.adapt(sample_reviews)

Let's run it on the same `X_example`, just to make sure the word IDs are larger now, since the vocabulary is bigger:

In [49]:
text_vectorization(X_example)

<tf.Tensor: shape=(2, 50), dtype=int64, numpy=
array([[  9,  14,   2,  64,  64,  12,   5, 257,   9,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  9,  13, 269, 531, 335,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

first 10 words in the vocabulary

In [50]:
text_vectorization.vocab[:10]

[b'<pad>', b'the', b'a', b'of', b'and', b'i', b'to', b'is', b'this', b'it']

These are the most common words in the reviews.

Now to build our model we will need to encode all these word IDs somehow. One approach is to create bags of words: for each review, and for each word in the vocabulary, we count the number of occurences of that word in the review. For example:

In [51]:
simple_example = tf.constant([[1, 3, 1, 0, 0] , [2, 2, 0, 0, 0]])
tf.reduce_sum(tf.one_hot(simple_example, 4), axis=1)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[2., 2., 0., 1.],
       [3., 0., 2., 0.]], dtype=float32)>

The first review has 2 times the word 0, 2 times the word 1, 0 times the word 2, and 1 time the word 3, so its bag-of-words representation is `[2, 2, 0, 1]`. Similarly, the second review has 3 times the word 0, 0 times the word 1, and so on. Let's wrap this logic in a small custom layer, and let's test it. We'll drop the counts for the word 0, since this corresponds to the `<pad>` token, which we don't care about.

In [52]:
class BagofWords(keras.layers.Layer):
  def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
    super().__init__(dtype = dtype, **kwargs)
    self.n_tokens = n_tokens
  def call(self, inputs):
    one_hot = tf.one_hot(inputs, self.n_tokens)
    return tf.reduce_sum(one_hot, axis=1)[:, 1:]

In [53]:
bag_of_words = BagofWords(n_tokens=4)

bag_of_words(simple_example)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2., 0., 1.],
       [0., 2., 0.]], dtype=float32)>

It works fine! Now let's create another `BagOfWord` with the right vocabulary size for our training set:

In [54]:
n_tokens = max_vocabulary_size + n_oov_buckets + 1 # add 1 for <pad>
bag_of_words = BagofWords(n_tokens)

Training model

In [56]:
model = keras.models.Sequential([
    text_vectorization,
    bag_of_words,
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(loss='binary_crossentropy', optimizer='nadam',
              metrics=['accuracy'])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
782/782 [==============================] - 28s 12ms/step - loss: 0.5821 - accuracy: 0.6835 - val_loss: 0.5072 - val_accuracy: 0.7398
Epoch 2/5
782/782 [==============================] - 10s 9ms/step - loss: 0.4626 - accuracy: 0.7763 - val_loss: 0.5084 - val_accuracy: 0.7415
Epoch 3/5
782/782 [==============================] - 9s 9ms/step - loss: 0.4146 - accuracy: 0.8120 - val_loss: 0.5244 - val_accuracy: 0.7347
Epoch 4/5
782/782 [==============================] - 9s 9ms/step - loss: 0.3402 - accuracy: 0.8623 - val_loss: 0.5359 - val_accuracy: 0.7365
Epoch 5/5
782/782 [==============================] - 9s 9ms/step - loss: 0.2548 - accuracy: 0.9123 - val_loss: 0.5859 - val_accuracy: 0.7257


We get about 74% accuracy on the validation set after just the first epoch, but after that the model makes no significant progress. For now the point is just to perform efficient preprocessing using `tf.data` and Keras preprocessing layers.

In [57]:
def compute_mean_embedding(inputs):
    not_pad = tf.math.count_nonzero(inputs, axis=-1)
    n_words = tf.math.count_nonzero(not_pad, axis=-1, keepdims=True)    
    sqrt_n_words = tf.math.sqrt(tf.cast(n_words, tf.float32))
    return tf.reduce_sum(inputs, axis=1) / sqrt_n_words

another_example = tf.constant([[[1., 2., 3.], [4., 5., 0.], [0., 0., 0.]],
                               [[6., 0., 0.], [0., 0., 0.], [0., 0., 0.]]])
compute_mean_embedding(another_example)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[3.535534 , 4.9497476, 2.1213205],
       [6.       , 0.       , 0.       ]], dtype=float32)>

In [58]:
tf.reduce_mean(another_example[0:1, :2], axis=1) * tf.sqrt(2.)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[3.535534 , 4.9497476, 2.1213202]], dtype=float32)>

In [59]:
tf.reduce_mean(another_example[1:2, :1], axis=1) * tf.sqrt(1.)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[6., 0., 0.]], dtype=float32)>

Now we're ready to train our final model. It's the same as before, except we replaced the `BagOfWords` layer with an `Embedding` layer followed by a `Lambda` layer that calls the `compute_mean_embedding` layer:

In [61]:
embedding_size = 20

model = keras.models.Sequential([
    text_vectorization,
    keras.layers.Embedding(input_dim=n_tokens,
                           output_dim=embedding_size,
                           mask_zero=True), # <pad> tokens => zero vectors
    keras.layers.Lambda(compute_mean_embedding),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])

Training the model and optimizing

In [62]:
model.compile(loss='binary_crossentropy', optimizer='nadam',metrics=['accuracy'])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
782/782 [==============================] - 26s 16ms/step - loss: 0.6099 - accuracy: 0.6513 - val_loss: 0.5106 - val_accuracy: 0.7401
Epoch 2/5
782/782 [==============================] - 9s 9ms/step - loss: 0.4961 - accuracy: 0.7542 - val_loss: 0.5150 - val_accuracy: 0.7392
Epoch 3/5
782/782 [==============================] - 9s 9ms/step - loss: 0.4785 - accuracy: 0.7649 - val_loss: 0.5073 - val_accuracy: 0.7436
Epoch 4/5
782/782 [==============================] - 15s 16ms/step - loss: 0.4709 - accuracy: 0.7648 - val_loss: 0.5102 - val_accuracy: 0.7439
Epoch 5/5
782/782 [==============================] - 15s 16ms/step - loss: 0.4653 - accuracy: 0.7680 - val_loss: 0.5077 - val_accuracy: 0.7430


The model is not better using embeddings

_Exercise: Use TFDS to load the same dataset more easily: `tfds.load("imdb_reviews")`._

In [63]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="imdb_reviews")
train_set, test_set = datasets["train"], datasets["test"]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ9RJXK/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ9RJXK/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ9RJXK/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [64]:
for example in train_set.take(1):
    print(example["text"])
    print(example["label"])

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)
